# More Data Processing with Pandas

## Merging DataFrames

In [4]:
#How to merge multiple DF, merging them horizontally or by concatening them vertically
#full outer join (database terminology) = union, in Venn diagramm = everyone in any circle
#inner join (database terminology) = intersection, in Venn diagramm = overlapping parts of each circle
import pandas as pd

staff_df = pd.DataFrame([{'Name':'Willy', 'Role':'Data Scientist'},
                         {'Name':'Tyty', 'Role':'Cybersecurity'},
                         {'Name':'Baptiste', 'Role':'Director of IT Department'}])
staff_df = staff_df.set_index('Name')
student_df = pd.DataFrame([{'Name':'Willy', 'School':'Engineering'},
                          {'Name':'Baptiste', 'School':'Business'},
                          {'Name':'Myriam', 'School':'Law'}])
student_df = student_df.set_index('Name')
print(staff_df.head())
print(student_df.head())

                               Role
Name                               
Willy                Data Scientist
Tyty                  Cybersecurity
Baptiste  Director of IT Department
               School
Name                 
Willy     Engineering
Baptiste     Business
Myriam            Law


In [5]:
#Exist some overlap : Willy and Baptiste are both student and staff.
#Impt: Both DF are indexed along the same index = Name
#union them => merge() passing in the DF on the left or right + telling if we want outer join,...
#Let's try the left and right indices as the joining col, with full outer join

pd.merge(staff_df,student_df, how='outer', left_index=True, right_index=True)

,Role,School
Name,,
Baptiste,Director of IT Department,Business
Myriam,NaN,Law
Tyty,Cybersecurity,NaN
Willy,Data Scientist,Engineering


In [6]:
#Let's try inner join
pd.merge(staff_df,student_df, how='inner', left_index=True, right_index=True)

,Role,School
Name,,
Willy,Data Scientist,Engineering
Baptiste,Director of IT Department,Business


In [7]:
#There are 2 other common use cases when merging => set addition. 1st: when we would want to get
#a list of all staff. 2st: the reverse with student list
pd.merge(staff_df,student_df, how='left', left_index=True, right_index=True)

,Role,School
Name,,
Willy,Data Scientist,Engineering
Tyty,Cybersecurity,NaN
Baptiste,Director of IT Department,Business


In [8]:
pd.merge(staff_df,student_df, how='right', left_index=True, right_index=True)

,Role,School
Name,,
Willy,Data Scientist,Engineering
Baptiste,Director of IT Department,Business
Myriam,NaN,Law


In [9]:
#Another way with the col => parameter 'on'. 1st let's remove the index
staff_df = staff_df.reset_index()
student_df = student_df.reset_index()

pd.merge(staff_df, student_df, how='right', on='Name')

,Name,Role,School
0,Willy,Data Scientist,Engineering
1,Baptiste,Director of IT Department,Business
2,Myriam,NaN,Law


In [11]:
#Using parameter 'on' => more common for merging DF
#What happened when we have conflicts btw the DF ? 

staff_df = pd.DataFrame([{'Name':'Willy', 'Role':'Data Scientist','Location':'Madrid'},
                         {'Name':'Tyty', 'Role':'Cybersecurity','Location':'Paris'},
                         {'Name':'Baptiste', 'Role':'Director of IT Department','Location':'Kiev'}])
student_df = pd.DataFrame([{'Name':'Willy', 'School':'Engineering','Location':'London'},
                          {'Name':'Baptiste', 'School':'Business','Location':'Korea'},
                          {'Name':'Myriam', 'School':'Law','Location':'Toronto'}])

#Students had their school in different countries. Merge function preserves this info, but appends an
# _x or _y to help diffrentiate btw which index went which col. _x => left DF, _y => right DF info.
#We want all staff info regardless of whether they are students or not
#If their are student => we want their student info => left join on the col Name
pd.merge(staff_df,student_df,how='left', on='Name')

,Name,Role,Location_x,School,Location_y
0,Willy,Data Scientist,Madrid,Engineering,London
1,Tyty,Cybersecurity,Paris,NaN,NaN
2,Baptiste,Director of IT Department,Kiev,Business,Korea


In [14]:
#Let's try multi-indexing & multiple col. Normal that first name for staff and student might overlap
#Not the same with last name => list of multiple col => join keys from both DF

staff_df = pd.DataFrame([{'First Name':'Willy','Last Name':'Sanchez','Role':'Data Scientist'},
                         {'First Name':'Tyty','Last Name':'Nguyen','Role':'Cybersecurity'},
                         {'First Name':'Baptiste','Last Name':'Hudy','Role':'Director of IT Department'}])
student_df = pd.DataFrame([{'First Name':'Willy','Last Name':'Drame','School':'Engineering'},
                          {'First Name':'Baptiste','Last Name':'Hudy','School':'Business'},
                          {'First Name':'Myriam','Last Name':'Silmes','School':'Law'}])
#Willy and Baptiste don't have the same last name in both DF
pd.merge(staff_df,student_df,how='inner',on=['First Name','Last Name'])

,First Name,Last Name,Role,School
0,Baptiste,Hudy,Director of IT Department,Business


In [15]:
#If we think of Merging = joining 'horizontally' => join similar values in a col found in 2 DF
#Concatenating => joinning 'vertically' => put DF on top or at bottom of each other
#Ex: Concatenating: A dataset that tracks some info over years + each years records is a separate CSV
#each CSV => same columns. We want all the data from all years record, together

In [16]:
#We'll use the US Department Of Education College Scorecard data => each US University's data on 
#student completion, debt, after-graduation income,... 
#Data is stored in separat CSV's and each CSV containing one year's record. 
#PB: CSV files => messy, I want to supress some Jupyter warning message => read_csv to ignore bad lines
#begin the cell with cell magic => '%%capture'

In [17]:
%%capture
df_2011 = pd.read_csv('MERGED2011_12_PP.csv', error_bad_lines=False)
df_2012 = pd.read_csv('MERGED2012_13_PP.csv', error_bad_lines=False)
df_2013 = pd.read_csv('MERGED2013_14_PP.csv', error_bad_lines=False)

In [18]:
df_2011.head(3)

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,COUNT_WNE_MALE1_P8,MD_EARN_WNE_MALE1_P8,GT_THRESHOLD_P10,MD_EARN_WNE_INC1_P10,MD_EARN_WNE_INC2_P10,MD_EARN_WNE_INC3_P10,MD_EARN_WNE_INDEP1_P10,MD_EARN_WNE_INDEP0_P10,MD_EARN_WNE_MALE0_P10,MD_EARN_WNE_MALE1_P10
0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100690,2503400,25034,Amridge University,Montgomery,AL,36117-3553,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
print(len(df_2011))
print(len(df_2012))
print(len(df_2013)) 
#Number of Schools increases normally

7746
7862
7869


In [20]:
frames = [df_2011, df_2012, df_2013]
pd.concat(frames)

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,COUNT_WNE_MALE1_P8,MD_EARN_WNE_MALE1_P8,GT_THRESHOLD_P10,MD_EARN_WNE_INC1_P10,MD_EARN_WNE_INC2_P10,MD_EARN_WNE_INC3_P10,MD_EARN_WNE_INDEP1_P10,MD_EARN_WNE_INDEP0_P10,MD_EARN_WNE_MALE0_P10,MD_EARN_WNE_MALE1_P10
0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100690,2503400,25034,Amridge University,Montgomery,AL,36117-3553,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100706,105500,1055,University of Alabama in Huntsville,Huntsville,AL,35899,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100724,100500,1005,Alabama State University,Montgomery,AL,36104-0271,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7864,48285703,157107,1571,Georgia Military College-Columbus Campus,Columbus,GA,31909,NaN,NaN,NaN,...,753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7865,48285704,157101,1571,Georgia Military College-Valdosta Campus,Valdosta,GA,31605,NaN,NaN,NaN,...,753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7866,48285705,157105,1571,Georgia Military College-Warner Robins Campus,Warner Robins,GA,31093,NaN,NaN,NaN,...,753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7867,48285706,157100,1571,Georgia Military College-Online,Milledgeville,GA,31061,NaN,NaN,NaN,...,753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
(len(df_2011)+len(df_2012)+len(df_2013))==len(pd.concat(frames))
#Concatenation works perfectly

True

In [23]:
#However we don't know what observations are from what year anymore => concat() has a param that 
#solves such pb => key param, we can set an extra lvl of indices + we pass in a list of keys that we
#want to correpond to the DF into keys param
pd.concat(frames,keys=['2011','2012','2013']) #Now we get the indices

UNITID    OPEID  OPEID6  \
2011 0       100654   100200    1002   
     1       100663   105200    1052   
     2       100690  2503400   25034   
     3       100706   105500    1055   
     4       100724   100500    1005   
...             ...      ...     ...   
2013 7864  48285703   157107    1571   
     7865  48285704   157101    1571   
     7866  48285705   157105    1571   
     7867  48285706   157100    1571   
     7868  48285707   157103    1571   

                                                  INSTNM            CITY  \
2011 0                          Alabama A & M University          Normal   
     1               University of Alabama at Birmingham      Birmingham   
     2                                Amridge University      Montgomery   
     3               University of Alabama in Huntsville      Huntsville   
     4                          Alabama State University      Montgomery   
...                                                  ...             ...   
2013 7864       Georgia Military College-Columbus Campus        Columbus   
     7865       Georgia Military College-Valdosta Campus        Valdosta   
     7866  Georgia Military College-Warner Robins Campus   Warner Robins   
     7867                Georgia Military College-Online   Milledgeville   
     7868        Georgia Military College-Stone Mountain  Stone Mountain   

          STABBR         ZIP  ACCREDAGENCY  INSTURL  NPCURL  ...  \
2011 0        AL       35762           NaN      NaN     NaN  ...   
     1        AL  35294-0110           NaN      NaN     NaN  ...   
     2        AL  36117-3553           NaN      NaN     NaN  ...   
     3        AL       35899           NaN      NaN     NaN  ...   
     4        AL  36104-0271           NaN      NaN     NaN  ...   
...          ...         ...           ...      ...     ...  ...   
2013 7864     GA       31909           NaN      NaN     NaN  ...   
     7865     GA       31605           NaN      NaN     NaN  ...   
     7866     GA       31093           NaN      NaN     NaN  ...   
     7867     GA       31061           NaN      NaN     NaN  ...   
     7868     GA       30083           NaN      NaN     NaN  ...   

           COUNT_WNE_MALE1_P8  MD_EARN_WNE_MALE1_P8  GT_THRESHOLD_P10  \
2011 0                    NaN                   NaN               NaN   
     1                    NaN                   NaN               NaN   
     2                    NaN                   NaN               NaN   
     3                    NaN                   NaN               NaN   
     4                    NaN                   NaN               NaN   
...                       ...                   ...               ...   
2013 7864                 753                   NaN               NaN   
     7865                 753                   NaN               NaN   
     7866                 753                   NaN               NaN   
     7867                 753                   NaN               NaN   
     7868                 753                   NaN               NaN   

           MD_EARN_WNE_INC1_P10  MD_EARN_WNE_INC2_P10  MD_EARN_WNE_INC3_P10  \
2011 0                      NaN                   NaN                   NaN   
     1                      NaN                   NaN                   NaN   
     2                      NaN                   NaN                   NaN   
     3                      NaN                   NaN                   NaN   
     4                      NaN                   NaN                   NaN   
...                         ...                   ...                   ...   
2013 7864                   NaN                   NaN                   NaN   
     7865                   NaN                   NaN                   NaN   
     7866                   NaN                   NaN                   NaN   
     7867                   NaN                   NaN                   NaN   
     7868                   NaN                   NaN   

In [24]:
#concat() has inner and outer method => if i'm concatenating 2 DF that don't have the same identical 
#col => choose outer method (but some cells => NaN). If i choose inner method => info will be dropped

## Pandas Idioms

In [25]:
#There are many ways the language can be used to solve a particular pb. But some are more appropriate
#than others => best solutions => celebrated as 'Idiomatic Python' and great ex on StackOverflow
#Pandas = sub-language of Python => own set of idioms as 'Vectorization' not using iterative loops if 
#i don't need to. Lot's of dev + users => used term __pandorable__ for these idioms.
#Here are some key to make my code more pandorable :)

In [26]:
import numpy as np
import timeit # timing functionnality from timeit module
df = pd.read_csv('census.csv')
df.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [28]:
cols = list(df.columns)
cols = [x.lower().strip() for x in cols]
df.columns = cols
df.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [31]:
#1st Pandas idioms = 'Chaining' => every method on an object returns a reference to that object
# => possibility to condense many diff oper on a DF.
#Here is a pandarable way to write code with method chaining. I'm going to pull out the race + sex 
#values+ to do so only for data which has native_country of Cuba
(df.where(df['native-country']==' Cuba')
   .dropna()
   .set_index(['race','sex'])
   .rename(columns={'occupation':'profession'}))

age          workclass  final-weight   education  \
race   sex                                                          
 Black  Female  28.0            Private      338409.0   Bachelors   
 White  Male    52.0            Private      276515.0   Bachelors   
        Male    47.0       Self-emp-inc      248145.0     5th-6th   
        Female  31.0            Private      243605.0   Bachelors   
        Female  41.0   Self-emp-not-inc      209344.0     HS-grad   
...              ...                ...           ...         ...   
        Male    37.0            Private      238433.0     1st-4th   
        Female  52.0            Private      222646.0        12th   
        Male    39.0   Self-emp-not-inc      251323.0         9th   
        Male    52.0       Self-emp-inc      230767.0   Bachelors   
        Male    48.0            Private      379883.0     Masters   

                education-num       marital-status          profession  \
race   sex                                                               
 Black  Female           13.0   Married-civ-spouse      Prof-specialty   
 White  Male             13.0   Married-civ-spouse       Other-service   
        Male              3.0   Married-civ-spouse    Transport-moving   
        Female           13.0              Widowed               Sales   
        Female            9.0   Married-civ-spouse               Sales   
...                       ...                  ...                 ...   
        Male              2.0   Married-civ-spouse    Transport-moving   
        Female            8.0            Separated   Machine-op-inspct   
        Male              5.0   Married-civ-spouse     Farming-fishing   
        Male             13.0   Married-civ-spouse     Exec-managerial   
        Male             14.0   Married-civ-spouse      Prof-specialty   

                   relationship  capital-gain  capital-loos  hour-per-week  \
race   sex                                                                   
 Black  Female             Wife           0.0           0.0           40.0   
 White  Male            Husband           0.0           0.0           40.0   
        Male            Husband           0.0           0.0           50.0   
        Female        Unmarried           0.0        1380.0           40.0   
        Female   Other-relative           0.0           0.0           40.0   
...                         ...           ...           ...            ...   
        Male            Husband           0.0           0.0           40.0   
        Female   Other-relative           0.0           0.0           40.0   
        Male     Other-relative           0.0           0.0           40.0   
        Male            Husband           0.0        1902.0           60.0   
        Male            Husband           0.0           0.0           40.0   

               native-country  income  
race   sex                             
 Black  Female           Cuba   <=50K  
 White  Male             Cuba   <=50K  
        Male             Cuba   <=50K  
        Female           Cuba   <=50K  
        Female           Cuba   <=50K  
...                       ...     ...  
        Male             Cuba   <=50K  
        Female           Cuba   <=50K  
        Male             Cuba   <=50K  
        Male             Cuba    >50K  
        Male             Cuba    >50K  

[95 rows x 13 columns]

In [33]:
#1st :we use where() on the DF and pass in a boolean mask => only true for the rows CI
#2nd :drop missing values by dft, 3rd: we set an index on the rslt, 4th: rename a column
#I could have done this in 1 line.
#See a non pandorable way to solve this issue
df = df[df['native-country']==' Cuba']
df.set_index(['race','sex'], inplace = True)
df.rename(columns={'occupation':'profession'})

age          workclass  final-weight   education  \
race   sex                                                         
 Black  Female   28            Private        338409   Bachelors   
 White  Male     52            Private        276515   Bachelors   
        Male     47       Self-emp-inc        248145     5th-6th   
        Female   31            Private        243605   Bachelors   
        Female   41   Self-emp-not-inc        209344     HS-grad   
...             ...                ...           ...         ...   
        Male     37            Private        238433     1st-4th   
        Female   52            Private        222646        12th   
        Male     39   Self-emp-not-inc        251323         9th   
        Male     52       Self-emp-inc        230767   Bachelors   
        Male     48            Private        379883     Masters   

                education-num       marital-status          profession  \
race   sex                                                               
 Black  Female             13   Married-civ-spouse      Prof-specialty   
 White  Male               13   Married-civ-spouse       Other-service   
        Male                3   Married-civ-spouse    Transport-moving   
        Female             13              Widowed               Sales   
        Female              9   Married-civ-spouse               Sales   
...                       ...                  ...                 ...   
        Male                2   Married-civ-spouse    Transport-moving   
        Female              8            Separated   Machine-op-inspct   
        Male                5   Married-civ-spouse     Farming-fishing   
        Male               13   Married-civ-spouse     Exec-managerial   
        Male               14   Married-civ-spouse      Prof-specialty   

                   relationship  capital-gain  capital-loos  hour-per-week  \
race   sex                                                                   
 Black  Female             Wife             0             0             40   
 White  Male            Husband             0             0             40   
        Male            Husband             0             0             50   
        Female        Unmarried             0          1380             40   
        Female   Other-relative             0             0             40   
...                         ...           ...           ...            ...   
        Male            Husband             0             0             40   
        Female   Other-relative             0             0             40   
        Male     Other-relative             0             0             40   
        Male            Husband             0          1902             60   
        Male            Husband             0             0             40   

               native-country  income  
race   sex                             
 Black  Female           Cuba   <=50K  
 White  Male             Cuba   <=50K  
        Male             Cuba   <=50K  
        Female           Cuba   <=50K  
        Female           Cuba   <=50K  
...                       ...     ...  
        Male             Cuba   <=50K  
        Female           Cuba   <=50K  
        Male             Cuba   <=50K  
        Male             Cuba    >50K  
        Male             Cuba    >50K  

[95 rows x 13 columns]

In [36]:
#Let's see which one is runs faster
def first_approach():
    global df
    return (df.where(df['native-country']==' Cuba')
   .dropna()
   .set_index(['race','sex'])
   .rename(columns={'occupation':'profession'}))

df = pd.read_csv('census.csv')
timeit.timeit(first_approach, number=10) #I run to 10 times the fonctions first_approach => mean(running time))

0.27769480000006297

In [37]:
#We use global variable df in the function. However changing a global var inside a function will
#modify the var even in gloval scope and i don't want that to happen in this case. Therefore, 
#for selecting native country as Cuba, i create a new df for those records

def second_approach():
    global df
    new_df = df[df['native-country']==' Cuba']
    new_df.set_index(['race','sex'], inplace = True)
    return new_df.rename(columns={'occupation':'profession'})

df = pd.read_csv('census.csv')
timeit.timeit(second_approach, number=10) #I run to 10 times the fonctions first_approach => mean(running time))


0.03191090000018448

In [38]:
#As we can see the 2nd approach is faster => particular exemple of a classic time readability trade off
#Lot's of ex on StackOverFlow. Choose the write coding depending on the case
#Let's see another Pandas idioms

In [39]:
#Python has map() function and Pandas got a similar function called applymap() => provide some funct
#should operate on each cell of DF and return set = DF. applymap() isn't often used. but map()
#on all rows of a DF => need to use the Pandas funct => apply (often used)

In [3]:
import pandas as pd #new dataset
df = pd.read_table('census2.txt',delimiter=',')
df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [4]:
df.columns

Index(['SUMLEV', 'REGION', 'DIVISION', 'STATE', 'COUNTY', 'STNAME', 'CTYNAME',
       'CENSUS2010POP', 'ESTIMATESBASE2010', 'POPESTIMATE2010',
       'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013',
       'POPESTIMATE2014', 'POPESTIMATE2015', 'NPOPCHG_2010', 'NPOPCHG_2011',
       'NPOPCHG_2012', 'NPOPCHG_2013', 'NPOPCHG_2014', 'NPOPCHG_2015',
       'BIRTHS2010', 'BIRTHS2011', 'BIRTHS2012', 'BIRTHS2013', 'BIRTHS2014',
       'BIRTHS2015', 'DEATHS2010', 'DEATHS2011', 'DEATHS2012', 'DEATHS2013',
       'DEATHS2014', 'DEATHS2015', 'NATURALINC2010', 'NATURALINC2011',
       'NATURALINC2012', 'NATURALINC2013', 'NATURALINC2014', 'NATURALINC2015',
       'INTERNATIONALMIG2010', 'INTERNATIONALMIG2011', 'INTERNATIONALMIG2012',
       'INTERNATIONALMIG2013', 'INTERNATIONALMIG2014', 'INTERNATIONALMIG2015',
       'DOMESTICMIG2010', 'DOMESTICMIG2011', 'DOMESTICMIG2012',
       'DOMESTICMIG2013', 'DOMESTICMIG2014', 'DOMESTICMIG2015', 'NETMIG2010',
       'NETMIG2011', 'NETMIG2012', 'NETMI

In [5]:
#1st : i need to create a funct which takes in a particular row of data => find min and max values 
#then return a new row => min_max()

def min_max(row):
    data = row[['POPESTIMATE2010',
                'POPESTIMATE2011',
                'POPESTIMATE2012',
                'POPESTIMATE2013',
                'POPESTIMATE2014',
                'POPESTIMATE2015']]
    return pd.Series({'min':np.min(data), 'max':np.max(data)})

In [7]:
import numpy as np
df.apply(min_max, axis='columns').head()

,min,max
0,4785161,4858979
1,54660,55347
2,183193,203709
3,26489,27341
4,22512,22861


In [8]:
#Instead of returning a separate series to display min and max => i will add 2 new col in the DF

def min_max(row):
    data = row[['POPESTIMATE2010',
                'POPESTIMATE2011',
                'POPESTIMATE2012',
                'POPESTIMATE2013',
                'POPESTIMATE2014',
                'POPESTIMATE2015']]
    row['max'] = np.max(data)
    row['min'] = np.min(data)
    return row

df.apply(min_max,axis='columns')
 

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,max,min
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594,4858979,4785161
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333,55347,54660
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499,203709,183193
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299,27341,26489
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861,22861,22512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3188,50,4,8,56,37,Wyoming,Sweetwater County,43806,43806,43593,...,-5.339774,-14.252889,-14.248864,1.255221,16.243199,-5.295460,-14.075283,-14.070195,45162,43593
3189,50,4,8,56,39,Wyoming,Teton County,21294,21294,21297,...,19.525929,14.143021,-0.564849,0.654527,2.408578,21.160658,16.308671,1.520747,23125,21297
3190,50,4,8,56,41,Wyoming,Uinta County,21118,21118,21102,...,-6.902954,-14.215862,-12.127022,-18.136812,-5.536861,-7.521840,-14.740608,-12.606351,21102,20822
3191,50,4,8,56,43,Wyoming,Washakie County,8533,8533,8545,...,-2.013502,-17.781491,1.682288,-11.990126,-1.182592,-2.250385,-18.020168,1.441961,8545,8316


In [10]:
rows = ['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014',
        'POPESTIMATE2015']
df.apply(lambda x: np.max(x[rows]),axis=1).head()

0    4858979
1      55347
2     203709
3      27341
4      22861
dtype: int64

In [11]:
#As we can see apply() => flexibility, we can pass into apply any customized function

## Group by

In [ ]:
#The idea behind groupby() funct is that is takes some DF, splits it into chunks based on some
#key value, then applies computation on those chunks, then combines the rslt into another DF
#In Pandas => is refered to as split-apply-combine pattern 

### Splitting 

In [13]:
df = pd.read_table('census2.txt',delimiter=',')
df = df[df['SUMLEV']==50]
df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,...,1.807375,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411


In [14]:
%%timeit -n 3

for state in df['STNAME'].unique():
    avg = np.average(df.where(df['STNAME']==state).dropna()['CENSUS2010POP'])
    print('counties in state ' + state + ' have an average population of ' + str(avg))

counties in state Alabama have an average population of 71339.34328358209
counties in state Alaska have an average population of 24490.724137931036
counties in state Arizona have an average population of 426134.4666666667
counties in state Arkansas have an average population of 38878.90666666667
counties in state California have an average population of 642309.5862068966
counties in state Colorado have an average population of 78581.1875
counties in state Connecticut have an average population of 446762.125
counties in state Delaware have an average population of 299311.3333333333
counties in state District of Columbia have an average population of 601723.0
counties in state Florida have an average population of 280616.5671641791
counties in state Georgia have an average population of 60928.63522012578
counties in state Hawaii have an average population of 272060.2
counties in state Idaho have an average population of 35626.86363636364
counties in state Illinois have an average populat

In [15]:
%%timeit -n 3
for group, frame in df.groupby('STNAME'):
    #groupby() returns a tuple, 1st value = key, here the state name
    #2nd one is projected DF that was found for that group
    avg = np.average(frame['CENSUS2010POP'])
    print('counties in state ' + group + ' have an average population of ' + str(avg))

counties in state Alabama have an average population of 71339.34328358209
counties in state Alaska have an average population of 24490.724137931036
counties in state Arizona have an average population of 426134.4666666667
counties in state Arkansas have an average population of 38878.90666666667
counties in state California have an average population of 642309.5862068966
counties in state Colorado have an average population of 78581.1875
counties in state Connecticut have an average population of 446762.125
counties in state Delaware have an average population of 299311.3333333333
counties in state District of Columbia have an average population of 601723.0
counties in state Florida have an average population of 280616.5671641791
counties in state Georgia have an average population of 60928.63522012578
counties in state Hawaii have an average population of 272060.2
counties in state Idaho have an average population of 35626.86363636364
counties in state Illinois have an average populat

In [16]:
#There is a huge diff of speed. 99% of the time => use groupby() on 1 or more col 
#I'll create some new funct and if the 1st letter of param is a capital M => return 0
# Capital Q => return 1 and return 2 otherwise

df = df.set_index('STNAME') #it's important to set the index to the col that i want to group 1st

def set_batch_number(item):
    if item[0]<'M':
        return 0
    if item[0]<'Q':
        return 1
    return 2
for group, frame in df.groupby(set_batch_number):
    print('There are ' + str(len(frame)) + ' records in group ' + str(group) + ' for processing.')

There are 1177 records in group 0 for processing.
There are 1134 records in group 1 for processing.
There are 831 records in group 2 for processing.


In [17]:
#I didn't pass a col name to groupby(), I set the index of DF to be STNAME
#If no col identifier is passed groupby() will auto use the index
#Now i'll use a dataset of housing from Airbnb

df = pd.read_table('listings.txt',delimiter=',')
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2818,Quiet Garden View Room & Super Fast WiFi,3159,Daniel,NaN,Oostelijk Havengebied - Indische Buurt,52.36435,4.94358,Private room,59,3,278,2020-02-14,1.90,1,152
1,20168,Studio with private bathroom in the centre 1,59484,Alexander,NaN,Centrum-Oost,52.36407,4.89393,Private room,200,1,339,2020-04-09,2.50,2,0
2,25428,"Lovely, 1 bed apt in Ctr (w.lift) -3/20-6/20(f...",56142,Joan,NaN,Centrum-West,52.37490,4.88487,Entire home/apt,125,14,5,2020-02-09,0.13,2,132
3,27886,"Romantic, stylish B&B houseboat in canal district",97647,Flip,NaN,Centrum-West,52.38761,4.89188,Private room,141,2,219,2020-07-25,1.94,1,66
4,28871,Comfortable double room,124245,Edwin,NaN,Centrum-West,52.36775,4.89092,Private room,75,2,336,2020-09-20,2.59,2,335


In [19]:
#How would I group by both of these columns ? 1st => multiindex then call groupby()
df = df.set_index(['room_type','price'])
# When multiindex => need to pass in the lvls we are interested in grouping by
for group,frame in df.groupby(level=(0,1)):
    print(group)


('Entire home/apt', 4)
('Entire home/apt', 10)
('Entire home/apt', 19)
('Entire home/apt', 20)
('Entire home/apt', 21)
('Entire home/apt', 24)
('Entire home/apt', 25)
('Entire home/apt', 29)
('Entire home/apt', 30)
('Entire home/apt', 32)
('Entire home/apt', 34)
('Entire home/apt', 35)
('Entire home/apt', 36)
('Entire home/apt', 37)
('Entire home/apt', 38)
('Entire home/apt', 39)
('Entire home/apt', 40)
('Entire home/apt', 41)
('Entire home/apt', 42)
('Entire home/apt', 43)
('Entire home/apt', 44)
('Entire home/apt', 45)
('Entire home/apt', 46)
('Entire home/apt', 47)
('Entire home/apt', 48)
('Entire home/apt', 49)
('Entire home/apt', 50)
('Entire home/apt', 51)
('Entire home/apt', 52)
('Entire home/apt', 53)
('Entire home/apt', 54)
('Entire home/apt', 55)
('Entire home/apt', 56)
('Entire home/apt', 57)
('Entire home/apt', 58)
('Entire home/apt', 59)
('Entire home/apt', 60)
('Entire home/apt', 61)
('Entire home/apt', 62)
('Entire home/apt', 63)
('Entire home/apt', 64)
('Entire home/apt

In [20]:
def grouping_fun(item):
    if item[1] == 100:
        return (item[0],'100')
    else:
        return (item[0],'not 100')
for group, frame in df.groupby(by=grouping_fun):
    print(group)

('Entire home/apt', '100')
('Entire home/apt', 'not 100')
('Hotel room', '100')
('Hotel room', 'not 100')
('Private room', '100')
('Private room', 'not 100')
('Shared room', '100')
('Shared room', 'not 100')


In [21]:
df.head()

id  \
room_type       price          
Private room    59      2818   
                200    20168   
Entire home/apt 125    25428   
Private room    141    27886   
                75     28871   

                                                                    name  \
room_type       price                                                      
Private room    59              Quiet Garden View Room & Super Fast WiFi   
                200         Studio with private bathroom in the centre 1   
Entire home/apt 125    Lovely, 1 bed apt in Ctr (w.lift) -3/20-6/20(f...   
Private room    141    Romantic, stylish B&B houseboat in canal district   
                75                               Comfortable double room   

                       host_id  host_name  neighbourhood_group  \
room_type       price                                            
Private room    59        3159     Daniel                  NaN   
                200      59484  Alexander                  NaN   
Entire home/apt 125      56142       Joan                  NaN   
Private room    141      97647       Flip                  NaN   
                75      124245      Edwin                  NaN   

                                                neighbourhood  latitude  \
room_type       price                                                     
Private room    59     Oostelijk Havengebied - Indische Buurt  52.36435   
                200                              Centrum-Oost  52.36407   
Entire home/apt 125                              Centrum-West  52.37490   
Private room    141                              Centrum-West  52.38761   
                75                               Centrum-West  52.36775   

                       longitude  minimum_nights  number_of_reviews  \
room_type       price                                                 
Private room    59       4.94358               3                278   
                200      4.89393               1                339   
Entire home/apt 125      4.88487              14                  5   
Private room    141      4.89188               2                219   
                75       4.89092               2                336   

                      last_review  reviews_per_month  \
room_type       price                                  
Private room    59     2020-02-14               1.90   
                200    2020-04-09               2.50   
Entire home/apt 125    2020-02-09               0.13   
Private room    141    2020-07-25               1.94   
                75     2020-09-20               2.59   

                       calculated_host_listings_count  availability_365  
room_type       price                                                    
Private room    59                                  1               152  
                200                                 2                 0  
Entire home/apt 125                                 2               132  
Private room    141                                 1                66  
                75                                  2               335

In [22]:
#Pandas dev => 3 broad categories of Data Processing to happen during apply step
# Aggregation of group data, Transformation of group data and Filtration of group data

### Aggregation

In [23]:
#most straight forward apply step = aggre of data => use agg() on groupby() object
#iterated through the groupby object, unpacking it into label (group name) and a DF
#with aff we can pass in a dict of the col we are interested in 

df = df.reset_index()
df.groupby('room_type').agg({'price':np.average}) # most recent Pandas version .agg() => deprecated
#might have to pass in custom fonction for similar effet


,price
room_type,
Entire home/apt,169.861687
Hotel room,135.120968
Private room,98.978435
Shared room,101.489796


In [24]:
#if there is NaN value => np.average doesn't ignore NaN 
df.groupby('room_type').agg({'price':np.nanmean})

,price
room_type,
Entire home/apt,169.861687
Hotel room,135.120968
Private room,98.978435
Shared room,101.489796


In [25]:
#we can extend this dict to aggregate by multiple funct or col
df.groupby('room_type').agg({'price':(np.average,np.nanstd),'number_of_reviews':np.nanmean})

price             number_of_reviews
                    average      nanstd           nanmean
room_type                                                
Entire home/apt  169.861687  160.081373         15.311744
Hotel room       135.120968  165.625230         49.250000
Private room      98.978435  113.509545         54.632588
Shared room      101.489796  123.831223         58.795918

In [26]:
#1st we're doing a group by on DF by col room_type => GroupBy object
#Then we're invoking agg() => apply 1 or more funct to the group DF and return a single row per DF/group
#When i called this funct => sent it 2 dict entries, each with the key indicating which col i want
#funct applied to. 1st col => tuple of 2 funct. groupby object => recognize the tuple and each funct
#in order the col. Rslt : hierarchical index, but there are col => don't show as an index per se.

### Transformation 

In [27]:
#agg() return a single value per col, 1 row per group. transformn() returns an object is the same size
#of the group => it broadcasts the funct we supply over the group
#Let's try to include the average price in a given group of room type, but preserve the DF shape

cols = ['room_type','price']
transform_df = df[cols].groupby('room_type').transform(np.nanmean)
transform_df.head()

,price
0,98.978435
1,98.978435
2,169.861687
3,98.978435
4,98.978435


In [28]:
#We can see that the index is same as the original df. 1st: let's rename the col in transform_df
transform_df.rename({'price':'mean_price'},axis='columns',inplace=True)
df = df.merge(transform_df,left_index=True,right_index=True)
df.head()

,room_type,price,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,mean_price
0,Private room,59,2818,Quiet Garden View Room & Super Fast WiFi,3159,Daniel,NaN,Oostelijk Havengebied - Indische Buurt,52.36435,4.94358,3,278,2020-02-14,1.90,1,152,98.978435
1,Private room,200,20168,Studio with private bathroom in the centre 1,59484,Alexander,NaN,Centrum-Oost,52.36407,4.89393,1,339,2020-04-09,2.50,2,0,98.978435
2,Entire home/apt,125,25428,"Lovely, 1 bed apt in Ctr (w.lift) -3/20-6/20(f...",56142,Joan,NaN,Centrum-West,52.37490,4.88487,14,5,2020-02-09,0.13,2,132,169.861687
3,Private room,141,27886,"Romantic, stylish B&B houseboat in canal district",97647,Flip,NaN,Centrum-West,52.38761,4.89188,2,219,2020-07-25,1.94,1,66,98.978435
4,Private room,75,28871,Comfortable double room,124245,Edwin,NaN,Centrum-West,52.36775,4.89092,2,336,2020-09-20,2.59,2,335,98.978435


In [29]:
df['mean_diff']=np.absolute(df['price']-df['mean_price'])
df['mean_diff'].head()

0     39.978435
1    101.021565
2     44.861687
3     42.021565
4     23.978435
Name: mean_diff, dtype: float64

### Filtering

In [30]:
#drop certain groups as part of the cleaning routines => filter() takes in a funct which it applies
#to each group df and return a boolean, depending upon whether that group should be included in the rslt.
df.groupby('room_type').filter(lambda x:np.nanmean(x['price'])>100)

,room_type,price,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,mean_price,mean_diff
2,Entire home/apt,125,25428,"Lovely, 1 bed apt in Ctr (w.lift) -3/20-6/20(f...",56142,Joan,NaN,Centrum-West,52.37490,4.88487,14,5,2020-02-09,0.13,2,132,169.861687,44.861687
6,Entire home/apt,160,41125,Amsterdam Center Entire Apartment,178515,Fatih,NaN,Centrum-West,52.37920,4.88432,4,89,2020-02-10,0.70,1,0,169.861687,9.861687
7,Entire home/apt,211,43109,Oasis in the middle of Amsterdam,188098,Aukje,NaN,Centrum-West,52.37366,4.88808,3,59,2020-01-02,0.46,1,362,169.861687,41.138313
8,Entire home/apt,65,43980,View into park / museum district (long/short s...,65041,Ym,NaN,Zuid,52.35735,4.86158,14,61,2018-02-18,0.48,2,164,169.861687,104.861687
9,Entire home/apt,150,46386,Cozy loft in central Amsterdam,207342,Joost,NaN,De Pijp - Rivierenbuurt,52.35198,4.90746,3,3,2018-01-03,0.02,1,0,169.861687,19.861687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17819,Entire home/apt,100,49078967,Cozy apartment next to center amsterdam,389785696,Danila,NaN,De Pijp - Rivierenbuurt,52.34147,4.90729,15,0,NaN,NaN,2,174,169.861687,69.861687
17821,Entire home/apt,200,49089176,"Prachtig, gerenoveerd appartement in de Pijp!",68481315,Julia,NaN,De Pijp - Rivierenbuurt,52.35416,4.88700,7,0,NaN,NaN,1,207,169.861687,30.138313
17822,Entire home/apt,112,49101225,Beautiful luxury loft close to city centre,23204045,Wes,NaN,Oud-Oost,52.35572,4.92440,1,0,NaN,NaN,1,86,169.861687,57.861687
17823,Entire home/apt,107,49102514,Cedo Nulli Dutch Holiday Boats,26206455,Dionne,NaN,Buitenveldert - Zuidas,52.33656,4.84835,1,0,NaN,NaN,1,335,169.861687,62.861687


In [33]:
#We see that the rslt is still indexed

### Applying

In [34]:
#Most common operation on groupby objects => apply()
df = pd.read_table('listings.txt',delimiter=',')
df=df[['room_type','price']]
df.head()

,room_type,price
0,Private room,59
1,Private room,200
2,Entire home/apt,125
3,Private room,141
4,Private room,75


In [35]:
def calc_mean_price(group):
    #group = df of whatever we have grouped by
    avg = np.nanmean(group['price'])
    group['price_mean'] = np.abs(avg-group['price'])
    return group
df.groupby('room_type').apply(calc_mean_price).head()

,room_type,price,price_mean
0,Private room,59,39.978435
1,Private room,200,101.021565
2,Entire home/apt,125,44.861687
3,Private room,141,42.021565
4,Private room,75,23.978435


# Scales

In [36]:
#Pandas doesn't capture the scale of the data. As a data scientist => 4 diff scales
#1: Ratio scale: units are equally spaced, maths ope of +-/* are valid, ex:height & weight
#interval scale: units are equally spaced but there's no true zero => basic maths ope not valid
#Ex: Temperature in Celcius, there's never an absence of temperature + 0 degre = meaningful value
#3: Ordinal scale: the order of the units = impt, but not evenly spaced. Ex: grades A-, A, A+ => very
#common in ML = challenge to work with
#4: Nominal scale, categorical of data, have no order with respect to one another. Ex: Teams of a sport
#very common in ML, exist 2 options of categ => binary categories

In [37]:
#Pandas has a number of interesting funct to deal with converting btw measurement scales.
#Nominal data => in Pandas = Categorical data, Pandas has a built in type for categorical D 
#We can set a col => categ D with astype() method, possible to change it to ordinal data

In [38]:
df = pd.DataFrame(['A+','A','A-','B+','B','B-','C+','C','C-','D+','D','D-'],
                 index=['excellent','excellent','excellent','good','good','good',
                       'ok','ok','ok','poor','poor','poor'],columns=['Grades'])
df

,Grades
excellent,A+
excellent,A
excellent,A-
good,B+
good,B
good,B-
ok,C+
ok,C
ok,C-
poor,D+


In [40]:
df.dtypes #type = object 

Grades    object
dtype: object

In [41]:
df['Grades'].astype('category').head() #12 categories
#the data = categorical + ordered

excellent    A+
excellent     A
excellent    A-
good         B+
good          B
Name: Grades, dtype: category
Categories (12, object): ['A', 'A+', 'A-', 'B', ..., 'C-', 'D', 'D+', 'D-']

In [42]:
#We can tell pandas that the D is ordered by 1st creating a new categ D type with the list of categ
#(in order) and ordered=True flag
my_categories=pd.CategoricalDtype(categories=['D-','D','D+','C-','C','C+','B-','B','B+','A-','A','A+'],
                                 ordered=True)
grades=df['Grades'].astype(my_categories)
grades.head()

excellent    A+
excellent     A
excellent    A-
good         B+
good          B
Name: Grades, dtype: category
Categories (12, object): ['D-' < 'D' < 'D+' < 'C-' ... 'B+' < 'A-' < 'A' < 'A+']

In [44]:
#Now Pandas aware of the 12 categories but also the order. It's ordering => can help with comparisons
# and boolean masking. We'll see that the lexicographical comparison returns rslt not intended
df[df['Grades']>'C'] #data not ordered categorical data

,Grades
ok,C+
ok,C-
poor,D+
poor,D
poor,D-


In [46]:
#Let's try with an ordered categorical data 
grades[grades>'C'] #it works

excellent    A+
excellent     A
excellent    A-
good         B+
good          B
good         B-
ok           C+
Name: Grades, dtype: category
Categories (12, object): ['D-' < 'D' < 'D+' < 'C-' ... 'B+' < 'A-' < 'A' < 'A+']

In [47]:
#Now We can use certain set of maths operators => min,max,... on the ordinal data
#Sometimes = useful to represent categorical values as each being a col with a T/F value as to whether
#the category applies. It's common in feature extraction => Data Mining course
#Boolean values => called dummy variables, Pandas has a built in funct call get_dummies which will
#convert the values of a single col to mult col of zeros and ones => indicating the presence of dummies

In [48]:
#Let's convert a scale from an interval or ratio scale (Numeric grade) into categorical D. 
#This might seem a bit counter intuitive bc I'm losing info => but commonly done 
#For instance, i we are visualizing the frequencies of categories => can be extrem useful approach
#histograms => regularly used with interval or ratio D.
#If i'm using a ML Classification approach of D => use categorical D, so reducing dim may be useful 
#just to apply a given technique. Pandas has a cut() funct => take as argue some array-like structure
#like a col Df or Series. But also, a nb of bins to be used, and all bins are kept at equal spacing. 

df = pd.read_table('census2.txt',delimiter=',')

#We reduce this to country data
df=df[df['SUMLEV']==50]
df=df.set_index('STNAME').groupby(level=0)['CENSUS2010POP'].agg(np.average)
df.head()


STNAME
Alabama        71339.343284
Alaska         24490.724138
Arizona       426134.466667
Arkansas       38878.906667
California    642309.586207
Name: CENSUS2010POP, dtype: float64

In [49]:
#If i want to make a 'bins' of each of these => use cut()
pd.cut(df,10) #10 bins

STNAME
Alabama                   (11706.087, 75333.413]
Alaska                    (11706.087, 75333.413]
Arizona                 (390320.176, 453317.529]
Arkansas                  (11706.087, 75333.413]
California              (579312.234, 642309.586]
Colorado                 (75333.413, 138330.766]
Connecticut             (390320.176, 453317.529]
Delaware                (264325.471, 327322.823]
District of Columbia    (579312.234, 642309.586]
Florida                 (264325.471, 327322.823]
Georgia                   (11706.087, 75333.413]
Hawaii                  (264325.471, 327322.823]
Idaho                     (11706.087, 75333.413]
Illinois                 (75333.413, 138330.766]
Indiana                   (11706.087, 75333.413]
Iowa                      (11706.087, 75333.413]
Kansas                    (11706.087, 75333.413]
Kentucky                  (11706.087, 75333.413]
Louisiana                 (11706.087, 75333.413]
Maine                    (75333.413, 138330.766]
Maryland     

In [50]:
#Cutting = just 1 way to built categories => other methods. Ex, cut gives an interval D, where the 
#spacing btw category is equally sized. But sometimes => want to form categ based on frequency
#So i want the nb of items in each bin to be the same, instead of spacing btw bins
#This depends on the shape of the D + my plan to do

# Pivot Table

In [51]:
#Pivot table = way of summarizing data in a DF => heavy use of the aggregation funct. 
#Pivot table = DF, where rows represent 1 var that i'm interested in, the col another and cell's
#some aggregate value. Pivot table => includes marginal val => sums for each col and row
#PT allows me to see relationship btw 2 var

In [54]:
df=pd.read_table('cwurData.txt',delimiter=',')
df.head(20)

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012
5,6,Princeton University,USA,5,8,14,2,53,33,26,NaN,101,82.50,2012
6,7,University of Oxford,United Kingdom,2,13,28,9,15,13,19,NaN,26,82.34,2012
7,8,Yale University,USA,6,14,31,12,14,6,15,NaN,66,79.14,2012
8,9,Columbia University,USA,7,23,21,10,13,12,14,NaN,5,78.86,2012
9,10,"University of California, Berkeley",USA,8,16,52,6,6,5,3,NaN,16,78.55,2012


In [70]:
#Let's create a new col called Rank_level => institutions with world ranking 1-100 are categorized
#as 1st tier and those 101-200 2nd tier, ranking 201-300 3rd tier after 301 is other top universities

In [65]:
def create_category(ranking):
    if (ranking >= 1) & (ranking <= 100):
        return 'First Tier Top University'
    elif (ranking >= 101) & (ranking <= 200):
        return 'Second Tier Top University'
    elif (ranking >= 201) & (ranking <= 300):
        return 'Third Tier Top University'
    else:
        return 'Other Top University'
    
df['Rank_Level'] = df['world_rank'].apply(lambda x: create_category(x))
df.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year,Rank_Level
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012,First Tier Top University
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012,First Tier Top University
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012,First Tier Top University
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012,First Tier Top University
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012,First Tier Top University


In [72]:
#PT allows us to pivot out one of these col a newcol headers and compare it against another col as row
#indices. let's compare rank lvl vs country of the universities and compare in terms of overall score

df.pivot_table(values='score',index='country',columns='Rank_Level',aggfunc=[np.mean]).head()

mean                       \
Rank_Level First Tier Top University Other Top University   
country                                                     
Argentina                        NaN            44.672857   
Australia                    47.9425            44.645750   
Austria                          NaN            44.864286   
Belgium                      51.8750            45.081000   
Brazil                           NaN            44.499706   

                                                                 
Rank_Level Second Tier Top University Third Tier Top University  
country                                                          
Argentina                         NaN                       NaN  
Australia                     49.2425                 47.285000  
Austria                           NaN                 47.066667  
Belgium                       49.0840                 46.746667  
Brazil                        49.5650                       NaN

In [73]:
df.pivot_table(values='score',index='country',columns='Rank_Level',aggfunc=[np.mean,np.max]).head()

mean                       \
Rank_Level First Tier Top University Other Top University   
country                                                     
Argentina                        NaN            44.672857   
Australia                    47.9425            44.645750   
Austria                          NaN            44.864286   
Belgium                      51.8750            45.081000   
Brazil                           NaN            44.499706   

                                                                 \
Rank_Level Second Tier Top University Third Tier Top University   
country                                                           
Argentina                         NaN                       NaN   
Australia                     49.2425                 47.285000   
Austria                           NaN                 47.066667   
Belgium                       49.0840                 46.746667   
Brazil                        49.5650                       NaN   

                                amax                       \
Rank_Level First Tier Top University Other Top University   
country                                                     
Argentina                        NaN                45.66   
Australia                      51.61                45.97   
Austria                          NaN                46.29   
Belgium                        52.03                46.21   
Brazil                           NaN                46.08   

                                                                 
Rank_Level Second Tier Top University Third Tier Top University  
country                                                          
Argentina                         NaN                       NaN  
Australia                       50.40                     47.47  
Austria                           NaN                     47.78  
Belgium                         49.73                     47.14  
Brazil                          49.82                       NaN

In [75]:
#We can also summarize the val within a given top lvl col. For ex: if i want to see an overall average
#for the country for the mean and i want the max of the max
df.pivot_table(values='score',index='country',columns='Rank_Level',aggfunc=[np.mean,np.max],
               margins=True).head()

mean                       \
Rank_Level First Tier Top University Other Top University   
country                                                     
Argentina                        NaN            44.672857   
Australia                    47.9425            44.645750   
Austria                          NaN            44.864286   
Belgium                      51.8750            45.081000   
Brazil                           NaN            44.499706   

                                                                            \
Rank_Level Second Tier Top University Third Tier Top University        All   
country                                                                      
Argentina                         NaN                       NaN  44.672857   
Australia                     49.2425                 47.285000  45.825517   
Austria                           NaN                 47.066667  45.139583   
Belgium                       49.0840                 46.746667  47.011000   
Brazil                        49.5650                       NaN  44.781111   

                                amax                       \
Rank_Level First Tier Top University Other Top University   
country                                                     
Argentina                        NaN                45.66   
Australia                      51.61                45.97   
Austria                          NaN                46.29   
Belgium                        52.03                46.21   
Brazil                           NaN                46.08   

                                                                        
Rank_Level Second Tier Top University Third Tier Top University    All  
country                                                                 
Argentina                         NaN                       NaN  45.66  
Australia                       50.40                     47.47  51.61  
Austria                           NaN                     47.78  47.78  
Belgium                         49.73                     47.14  52.03  
Brazil                          49.82                       NaN  49.82

In [76]:
#PT = multi-lvl DF and i can access series or cells in that DF in similar way as a regular DF
new_df = df.pivot_table(values='score',index='country',columns='Rank_Level',aggfunc=[np.mean,np.max],
               margins=True)
print(new_df.index)
print(new_df.columns)

Index(['Argentina', 'Australia', 'Austria', 'Belgium', 'Brazil', 'Bulgaria',
       'Canada', 'Chile', 'China', 'Colombia', 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Egypt', 'Estonia', 'Finland', 'France',
       'Germany', 'Greece', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Iran',
       'Ireland', 'Israel', 'Italy', 'Japan', 'Lebanon', 'Lithuania',
       'Malaysia', 'Mexico', 'Netherlands', 'New Zealand', 'Norway', 'Poland',
       'Portugal', 'Puerto Rico', 'Romania', 'Russia', 'Saudi Arabia',
       'Serbia', 'Singapore', 'Slovak Republic', 'Slovenia', 'South Africa',
       'South Korea', 'Spain', 'Sweden', 'Switzerland', 'Taiwan', 'Thailand',
       'Turkey', 'USA', 'Uganda', 'United Arab Emirates', 'United Kingdom',
       'Uruguay', 'All'],
      dtype='object', name='country')
MultiIndex([('mean',  'First Tier Top University'),
            ('mean',       'Other Top University'),
            ('mean', 'Second Tier Top University'),
            ('mean',  'Thir

In [77]:
#We can see the col are hierarchical. Top lvl colo indices 2 categories: mean and max, and the lower 
#lvl col indices have 4 categories => 4 rank lvls
#Let's query the average scores of the First Tier Top University lvls in each country => 2 DF projections
#1st for the mean and 2nd for the top tier
new_df['mean']['First Tier Top University'].head()

country
Argentina        NaN
Australia    47.9425
Austria          NaN
Belgium      51.8750
Brazil           NaN
Name: First Tier Top University, dtype: float64

In [82]:
#Output = Series object. Remember that when projecting a single col of values out of a DF => Series
type(new_df['mean']['First Tier Top University'])

pandas.core.series.Series

In [84]:
#Let's find the country that has the max average score on First Tier Top University lvl =>
#idxmax()
new_df['mean']['First Tier Top University'].idxmax()
#idxmax() isn't special for PT => built in funct to Series

'United Kingdom'

In [85]:
#If i want to achieve a diff shape of my PT => can do so with stack and unstack funct
#Stacking = pivoting the lowermost col index to become innermost row index
#Unstacking = reverse

new_df.head()

mean                       \
Rank_Level First Tier Top University Other Top University   
country                                                     
Argentina                        NaN            44.672857   
Australia                    47.9425            44.645750   
Austria                          NaN            44.864286   
Belgium                      51.8750            45.081000   
Brazil                           NaN            44.499706   

                                                                            \
Rank_Level Second Tier Top University Third Tier Top University        All   
country                                                                      
Argentina                         NaN                       NaN  44.672857   
Australia                     49.2425                 47.285000  45.825517   
Austria                           NaN                 47.066667  45.139583   
Belgium                       49.0840                 46.746667  47.011000   
Brazil                        49.5650                       NaN  44.781111   

                                amax                       \
Rank_Level First Tier Top University Other Top University   
country                                                     
Argentina                        NaN                45.66   
Australia                      51.61                45.97   
Austria                          NaN                46.29   
Belgium                        52.03                46.21   
Brazil                           NaN                46.08   

                                                                        
Rank_Level Second Tier Top University Third Tier Top University    All  
country                                                                 
Argentina                         NaN                       NaN  45.66  
Australia                       50.40                     47.47  51.61  
Austria                           NaN                     47.78  47.78  
Belgium                         49.73                     47.14  52.03  
Brazil                          49.82                       NaN  49.82

In [86]:
#Let's try stacking
new_df = new_df.stack()
new_df.head()

mean   amax
country   Rank_Level                                  
Argentina Other Top University        44.672857  45.66
          All                         44.672857  45.66
Australia First Tier Top University   47.942500  51.61
          Other Top University        44.645750  45.97
          Second Tier Top University  49.242500  50.40

In [89]:
#Let's try unstacking it
new_df.unstack().head()

mean                       \
Rank_Level First Tier Top University Other Top University   
country                                                     
All                        58.350675            44.738871   
Argentina                        NaN            44.672857   
Australia                  47.942500            44.645750   
Austria                          NaN            44.864286   
Belgium                    51.875000            45.081000   

                                                                            \
Rank_Level Second Tier Top University Third Tier Top University        All   
country                                                                      
All                          49.06545                 46.843450  47.798395   
Argentina                         NaN                       NaN  44.672857   
Australia                    49.24250                 47.285000  45.825517   
Austria                           NaN                 47.066667  45.139583   
Belgium                      49.08400                 46.746667  47.011000   

                                amax                       \
Rank_Level First Tier Top University Other Top University   
country                                                     
All                           100.00                46.34   
Argentina                        NaN                45.66   
Australia                      51.61                45.97   
Austria                          NaN                46.29   
Belgium                        52.03                46.21   

                                                                         
Rank_Level Second Tier Top University Third Tier Top University     All  
country                                                                  
All                             51.29                     47.93  100.00  
Argentina                         NaN                       NaN   45.66  
Australia                       50.40                     47.47   51.61  
Austria                           NaN                     47.78   47.78  
Belgium                         49.73                     47.14   52.03

In [90]:
#That restore our DF to its original shape. What happen if i unstacked twice 
new_df.unstack().unstack().head()

      Rank_Level                 country  
mean  First Tier Top University  All          58.350675
                                 Argentina          NaN
                                 Australia    47.942500
                                 Austria            NaN
                                 Belgium      51.875000
dtype: float64

In [91]:
#I end up unstacking all the way to just a single col => series. This col = 'value' => denoted by the
#hierarchichal index of ope, rank and country

# Date/Time Functionnality

### Timestamp

In [92]:
# Pandas has 4 main time related classes. Timestamp, DateTimeIndex, Period and PeriodIndex.
# Timestamp represents a single timestamp and associates values with points in time
# Ex: 9/1/2019 10:05AM = timestamp, it's interchangeable with Python's datetime in most cases

pd.Timestamp('26/5/2022 12:22PM')

Timestamp('2022-05-26 12:22:00')

In [93]:
pd.Timestamp(2022,5,22,0,0)

Timestamp('2022-05-22 00:00:00')

In [94]:
#Timestamp got some useful attributes, such as isoweekday() which shows the weekday of the timestamp
#1 reprensent Monday and 7 Sunday
pd.Timestamp(2022,5,22,0,0).isoweekday()

7

In [95]:
pd.Timestamp(2022,5,22,2,33).second

0

### Period

In [96]:
#We're interested in a specific point in time and instead wanted a span of time.
#Period represents a single time span, such as a specific day or month
pd.Period('1/2022')

Period('2022-01', 'M')

In [97]:
#M for Month. Period print that out that the granularity of the period is M
pd.Period('3/5/2022')

Period('2022-03-05', 'D')

In [98]:
pd.Period('5/2022') + 5

Period('2022-10', 'M')

In [99]:
pd.Period('3/5/2022') - 2

Period('2022-03-03', 'D')

In [100]:
#The key here is that the period object encapsulates the granularity for arithmeticç

### DatetimeIndex and PeriodIndex

In [101]:
#The index of a timestamp is DateTimeIndex. 
t1 = pd.Series(list('abc'),[pd.Timestamp('2022-05-22'),pd.Timestamp('2022-04-12'),
                            pd.Timestamp('2022-03-22')])
t1

2022-05-22    a
2022-04-12    b
2022-03-22    c
dtype: object

In [102]:
type(t1.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [103]:
#We can create a period-based index 
t2 = pd.Series(list('def'),[pd.Period('2022-05'),pd.Period('2022-04'),
                            pd.Period('2022-03')])
t2

2022-05    d
2022-04    e
2022-03    f
Freq: M, dtype: object

In [104]:
type(t2)

pandas.core.series.Series

### Converting to Datetime

In [112]:
d1 = ['2 June 2019', 'Aug 16, 2017', '2017-04-23', '7/12/18'] #bunch of different format
ts3 = pd.DataFrame(np.random.randint(10,100,(4,2)),index=d1,columns=list('ab'))
ts3

,a,b
2 June 2019,20,13
"Aug 16, 2017",52,70
2017-04-23,31,55
7/12/18,69,36


In [113]:
#to_datetime => pandas will try to convert these to datetime and put them in a standard format 
ts3.index=pd.to_datetime(ts3.index)
ts3

,a,b
2019-06-02,20,13
2017-08-16,52,70
2017-04-23,31,55
2018-07-12,69,36


In [115]:
#to_datetime => has options to change the date parse order. For ex we can pass in the argument 
#dayfirst = True to parse the date in European date.

pd.to_datetime('4.7.12', dayfirst=True)

Timestamp('2012-07-04 00:00:00')

In [116]:
#Timedeltas are diff in times. This isn't as a period, but conceptually similar. 
#For ex: If i want to take the diff btw Sep 3rd and Sep 1st => Timedelta of 2 days 
pd.Timestamp('9/3/2019')-pd.Timestamp('9/1/2019')

Timedelta('2 days 00:00:00')

In [117]:
#We can also do something like, what the date and time is for 12 days and 3 hours past Sept 2nd
pd.Timestamp('9/2/2019 8:10AM') + pd.Timedelta('12D 3H')

Timestamp('2019-09-14 11:10:00')

### Offset

In [118]:
#Offset is similar to timedelta => follows specific calendar duration rules.
#it allows flexibility in terms of types of time intervals

pd.Timestamp('9/4/2019').weekday()

2

In [119]:
pd.Timestamp('9/4/2019') + pd.offsets.Week() #add a week

Timestamp('2019-09-11 00:00:00')

In [120]:
pd.Timestamp('9/4/2019') + pd.offsets.MonthEnd()

Timestamp('2019-09-30 00:00:00')

# Working with Dates in a Dataframe

In [123]:
#We want to look at nine measurements, taken bi-weekly, every Sunday in October 2016. Using date_range
#we can create a DateTimeIndex. In date_range we have to either specify the start or end date. 
#By default, date is considered the start date. We have to specify nb of periods and a frequency
#Here we set it to '2W-SUN' => means biweekly on Sunday 

dates =pd.date_range('10-01-2019',periods=9, freq='2W-SUN')
dates

DatetimeIndex(['2019-10-06', '2019-10-20', '2019-11-03', '2019-11-17',
               '2019-12-01', '2019-12-15', '2019-12-29', '2020-01-12',
               '2020-01-26'],
              dtype='datetime64[ns]', freq='2W-SUN')

In [125]:
pd.date_range('10-01-2019',periods=9, freq='B') #Business Day

DatetimeIndex(['2019-10-01', '2019-10-02', '2019-10-03', '2019-10-04',
               '2019-10-07', '2019-10-08', '2019-10-09', '2019-10-10',
               '2019-10-11'],
              dtype='datetime64[ns]', freq='B')

In [127]:
pd.date_range('10-01-2019',periods=12, freq='QS-JUN') #Quaterly

DatetimeIndex(['2019-12-01', '2020-03-01', '2020-06-01', '2020-09-01',
               '2020-12-01', '2021-03-01', '2021-06-01', '2021-09-01',
               '2021-12-01', '2022-03-01', '2022-06-01', '2022-09-01'],
              dtype='datetime64[ns]', freq='QS-JUN')

In [129]:
dates = pd.date_range('10-01-2019', periods=9, freq='2W-SUN')
df = pd.DataFrame({'Count1':100 + np.random.randint(-5,10,9).cumsum(),
                  'Count2': 120 + np.random.randint(-5,10,9)},index=dates)
df

,Count1,Count2
2019-10-06,102,117
2019-10-20,100,126
2019-11-03,95,122
2019-11-17,104,117
2019-12-01,106,124
2019-12-15,103,123
2019-12-29,110,118
2020-01-12,105,120
2020-01-26,109,118


In [132]:
df.index.day_name()

Index(['Sunday', 'Sunday', 'Sunday', 'Sunday', 'Sunday', 'Sunday', 'Sunday',
       'Sunday', 'Sunday'],
      dtype='object')

In [134]:
df.diff() #to see the difference btw each date's value

,Count1,Count2
2019-10-06,NaN,NaN
2019-10-20,-2.0,9.0
2019-11-03,-5.0,-4.0
2019-11-17,9.0,-5.0
2019-12-01,2.0,7.0
2019-12-15,-3.0,-1.0
2019-12-29,7.0,-5.0
2020-01-12,-5.0,2.0
2020-01-26,4.0,-2.0


In [135]:
#Let's try to know the mean count is for each month in our DataFrame => use resample. 
#Converting from higher freq to lower freq = downsampling 
df.resample('M').mean() #resample to a month week 

,Count1,Count2
2019-10-31,101.000000,121.500000
2019-11-30,99.500000,119.500000
2019-12-31,106.333333,121.666667
2020-01-31,107.000000,119.000000


In [141]:
#Let's try some indexing and slicing. We can use partial string indexing to find values from a 
#particular year.
df['2019']

C:\Users\wsanchez\AppData\Local\Temp/ipykernel_15584/290152354.py:3: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  df['2019']


,Count1,Count2
2019-10-06,102,117
2019-10-20,100,126
2019-11-03,95,122
2019-11-17,104,117
2019-12-01,106,124
2019-12-15,103,123
2019-12-29,110,118


In [144]:
df['2019-12']

C:\Users\wsanchez\AppData\Local\Temp/ipykernel_15584/2940089508.py:1: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  df['2019-12']


,Count1,Count2
2019-12-01,106,124
2019-12-15,103,123
2019-12-29,110,118


In [140]:
df['2019-12':]

,Count1,Count2
2019-12-01,106,124
2019-12-15,103,123
2019-12-29,110,118
2020-01-12,105,120
2020-01-26,109,118
